# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/openair2/spec_ingestor.py F1AP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-CU crashes when receiving F1AP Setup Response.
Repro Steps:
1) The attacker intercepts a legitimate UE Context Modification request.
2) Modify the RNTI in the IE to an incorrect value and forward it to the gNB-CU.
3) For gNB-DUs, the attacker waits for UE release and immediately sends a DLRRCMessage Transfer.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "gNB-CU crashes when receiving F1AP Setup Response; crafted UE Context Modification with incorrect RNTI; DU may send DL RRC Message Transfer right after UE release.",
    "interface_guess": ["F1AP", "RRC"],
    "procedure_guess": ["F1 Setup", "UE Context Modification", "DL RRC Message Transfer", "UE Context Release"],
    "components_involved": ["gNB-CU", "gNB-DU", "UE"],
    "key_ids": {
      "transaction_id": "unknown",
      "cu_ue_f1ap_id": "unknown",
      "du_ue_f1ap_id": "unknown",
      "du_id": "unknown",
      "rnti": "modified/incorrect",
      "pci": "unknown",
      "served_cell_list": []
    },
    "signals_or_timers": ["Transaction ID", "gNB-CU UE F1AP ID", "gNB-DU UE F1AP ID", "RRC-Container"]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": ["F1 Setup Response", "Transaction ID", "CU->DU", "Cells to be Activated"],
      "title": ["TS 38.473", "9.2.1.5", "F1 SETUP RESPONSE"]
    },
    {
      "Keywords": ["DL RRC Message Transfer", "gNB-CU UE F1AP ID", "gNB-DU UE F1AP ID", "RRC-Container"],
      "title": ["TS 38.473", "8.4.2", "9.2.3.2", "DL RRC MESSAGE TRANSFER"]
    },
    {
      "Keywords": ["UE Context Modification Required", "Confirm", "IDs", "Cause"],
      "title": ["TS 38.473", "9.2.2.10", "9.2.2.11", "UE CONTEXT MODIFICATION"]
    },
    {
      "Keywords": ["Transaction ID", "procedure scoping"],
      "title": ["TS 38.473", "9.3.1.23", "Transaction ID"]
    }
  ],
  "expected_behaviour": "CU sends F1 SETUP RESPONSE to DU with Transaction ID and optional cell info; DU processes it. DL RRC MESSAGE TRANSFER from CU to DU shall carry both UE F1AP IDs and an RRC-Container to deliver L3 payload without interpretation. UE Context Modification is CU-initiated (Request) with DU responding (Confirm/Refuse); IDs consistently bind the UE context. None of these should cause CU to crash."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "F1AP/f1ap_du_interface_management.c",
      "function_name": "DU_handle_F1_SETUP_RESPONSE",
      "reason": "DU-side handler for F1 Setup Response; relevant to crash scenario around Setup Response processing."
    },
    {
      "path": "F1AP/lib/f1ap_interface_management.c",
      "function_name": "decode_f1ap_setup_response",
      "reason": "Decodes F1 Setup Response including Transaction ID and optional cell IEs."
    },
    {
      "path": "F1AP/f1ap_du_rrc_message_transfer.c",
      "function_name": "DU_handle_DL_RRC_MESSAGE_TRANSFER",
      "reason": "Handles DL RRC Message Transfer (CU→DU); uses UE F1AP IDs to dispatch."
    },
    {
      "path": "F1AP/lib/f1ap_rrc_message_transfer.c",
      "function_name": "decode_dl_rrc_message_transfer",
      "reason": "Parses mandatory gNB-CU/gNB-DU UE F1AP IDs and RRC-Container from DL RRC Message Transfer."
    },
    {
      "path": "F1AP/f1ap_du_ue_context_management.c",
      "function_name": "DU_handle_UE_CONTEXT_MODIFICATION_REQUEST",
      "reason": "Processes CU-initiated UE Context Modification Request at DU, then responds."
    },
    {
      "path": "F1AP/lib/f1ap_ue_context.c",
      "function_name": "decode_ue_context_mod_req",
      "reason": "Decodes UE Context Modification Request; validates and binds F1AP UE IDs."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "spec",
      "source": "TS 38.473",
      "location": "§9.2.1.5 F1 SETUP RESPONSE",
      "text": "This message is sent by the gNB-CU to transfer information associated to an F1-C interface instance. Direction: gNB-CU → gNB-DU. Includes Message Type and Transaction ID (9.3.1.23), optional Cells to be Activated List, and gNB-CU RRC version."
    },
    {
      "kind": "spec",
      "source": "TS 38.473",
      "location": "§8.4.2 DL RRC Message Transfer – Successful operation",
      "text": "CU initiates DL RRC MESSAGE TRANSFER. If a UE-associated logical F1-connection exists, the message shall contain the gNB-DU UE F1AP ID. If not, DU establishes the association upon reception."
    },
    {
      "kind": "spec",
      "source": "TS 38.473",
      "location": "§9.2.3.2 DL RRC MESSAGE TRANSFER",
      "text": "Mandatory IEs include gNB-CU UE F1AP ID (9.3.1.4), gNB-DU UE F1AP ID (9.3.1.5), SRB ID (9.3.1.7), and RRC-Container (9.3.1.6)."
    },
    {
      "kind": "spec",
      "source": "TS 38.473",
      "location": "§9.2.2.10/§9.2.2.11 UE CONTEXT MODIFICATION (Required/Confirm)",
      "text": "UE Context Modification Required is DU→CU; Confirm/Refuse are CU→DU. Both carry gNB-CU UE F1AP ID and gNB-DU UE F1AP ID."
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_du_interface_management.c",
      "location": "162-175",
      "text": "DU_handle_F1_SETUP_RESPONSE: decode_f1ap_setup_response(&resp); on success calls f1_setup_response(&resp); frees resp."
    },
    {
      "kind": "code",
      "source": "F1AP/lib/f1ap_interface_management.c",
      "location": "1005-1009",
      "text": "Comment: F1 Setup Response encoding (9.2.1.5 of 3GPP TS 38.473) – gNB-CU → gNB-DU."
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_du_rrc_message_transfer.c",
      "location": "58-69",
      "text": "DU_handle_DL_RRC_MESSAGE_TRANSFER: if (!decode_dl_rrc_message_transfer(pdu,&msg)) LOG_E(...); dl_rrc_message_transfer(&msg); free_dl_rrc_message_transfer(&msg);"
    },
    {
      "kind": "code",
      "source": "F1AP/lib/f1ap_rrc_message_transfer.c",
      "location": "390-405",
      "text": "decode_dl_rrc_message_transfer: finds IEs; sets out->gNB_CU_ue_id and out->gNB_DU_ue_id from mandatory IEs."
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_du_ue_context_management.c",
      "location": "125-136",
      "text": "DU_handle_UE_CONTEXT_MODIFICATION_REQUEST: decode_ue_context_mod_req(pdu,&req); ue_context_modification_request(&req); free_ue_context_mod_req(&req);"
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {"from": "gNB-DU", "to": "gNB-CU", "message": "F1 SETUP REQUEST"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "F1 SETUP RESPONSE", "postcond": "F1 association ready"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UE CONTEXT MODIFICATION REQUEST", "precond": "UE context exists"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE CONTEXT MODIFICATION CONFIRM"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "DL RRC MESSAGE TRANSFER", "precond": "gNB-CU/DU UE F1AP IDs known"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE CONTEXT RELEASE REQUEST", "precond": "context not retrievable or release flow"}
  ],
  "state_machine": {
    "states": ["Init", "F1SetupPending", "F1SetupEstablished", "UEContextActive", "DLTransfer", "ReleasePending", "Closed"],
    "transitions": [
      {"from": "Init", "to": "F1SetupPending", "on": "send F1 SETUP REQUEST"},
      {"from": "F1SetupPending", "to": "F1SetupEstablished", "on": "recv F1 SETUP RESPONSE", "guard": "Transaction ID valid"},
      {"from": "F1SetupEstablished", "to": "UEContextActive", "on": "UE CONTEXT MODIFICATION REQUEST accepted", "guard": "UE F1AP IDs match"},
      {"from": "UEContextActive", "to": "DLTransfer", "on": "recv DL RRC MESSAGE TRANSFER", "guard": "IDs resolve to context"},
      {"from": "UEContextActive", "to": "ReleasePending", "on": "UE release initiated"},
      {"from": "ReleasePending", "to": "Closed", "on": "release complete"}
    ]
  },
  "assumptions": [
    "CU should never receive F1 SETUP RESPONSE; it originates from CU (to DU)",
    "Invalid RNTI in RRC payload must not crash CU; F1AP context resolution uses F1AP IDs",
    "Unknown/old gNB-DU UE F1AP ID should trigger graceful handling per spec"
  ],
  "questions": [
    "Exact CU crash site (function/line)?",
    "Are unknown UE F1AP IDs being validated before dereference?",
    "Is there any path where CU mistakenly treats incoming F1 SETUP RESPONSE as inbound?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "F1 Setup Response direction is CU→DU (not received by CU)",
      "result": "FAIL",
      "evidence": ["TS 38.473 §9.2.1.5 F1 SETUP RESPONSE", "Sequence shows crash on CU when \"receiving\" response"]
    },
    {
      "name": "DL RRC Message Transfer binds gNB-CU/gNB-DU UE F1AP IDs",
      "result": "PASS",
      "evidence": ["TS 38.473 §9.2.3.2 DL RRC MESSAGE TRANSFER", "F1AP/lib/f1ap_rrc_message_transfer.c:390-405"]
    },
    {
      "name": "UE Context Modification pairing (CU Request ↔ DU Confirm/Refuse)",
      "result": "PASS",
      "evidence": ["TS 38.473 §9.2.2.10/§9.2.2.11", "F1AP/f1ap_du_ue_context_management.c:125-136"]
    },
    {
      "name": "Transaction ID scoping for F1 Setup", 
      "result": "UNKNOWN",
      "evidence": ["TS 38.473 §9.3.1.23 Transaction ID", "Decode path present: F1AP/lib/f1ap_interface_management.c:1005-1009"]
    },
    {
      "name": "Graceful handling of unknown/old UE F1AP IDs in DL RRC path",
      "result": "UNKNOWN",
      "evidence": ["TS 38.473 §8.4.2 Successful operation", "F1AP/f1ap_du_rrc_message_transfer.c:58-69"]
    }
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "CU should not receive an F1 SETUP RESPONSE; crash suggests misrouted handling or missing directionality/validation checks. ID-bearing procedures (DL RRC, UE Context Modification) appear correctly decoded, so the failure likely lies in message routing or error handling rather than spec nonconformance."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "TS 38.473 §9.2.1.5 F1 SETUP RESPONSE",
      "TS 38.473 §8.4.2 DL RRC Message Transfer (Successful operation)",
      "TS 38.473 §9.2.3.2 DL RRC MESSAGE TRANSFER",
      "TS 38.473 §9.2.2.10/§9.2.2.11 UE CONTEXT MODIFICATION",
      "TS 38.473 §9.3.1.23 Transaction ID"
    ],
    "code_citations": [
      "F1AP/f1ap_du_interface_management.c:162-175",
      "F1AP/lib/f1ap_interface_management.c:1005-1009",
      "F1AP/f1ap_du_rrc_message_transfer.c:58-69",
      "F1AP/lib/f1ap_rrc_message_transfer.c:390-405",
      "F1AP/f1ap_du_ue_context_management.c:125-136"
    ],
    "expected_vs_observed": "Expected: CU sends F1 SETUP RESPONSE to DU; DU processes it; DL RRC and UE Context Modification use UE F1AP IDs to resolve context without crashing on malformed RRC payloads. Observed: CU reportedly crashes when \"receiving\" F1 SETUP RESPONSE, indicating misrouted handling or missing directionality/validation, not a spec allowance."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "By spec, F1 SETUP RESPONSE is CU→DU. A CU crash on \"receiving\" this message indicates misrouted handling or missing directionality/validation. DL RRC and UE Context Modification paths correctly rely on UE F1AP IDs; a malformed RNTI in the RRC payload should not crash CU.",
    "citations": {
      "spec": [
        "TS 38.473 §9.2.1.5 F1 SETUP RESPONSE",
        "TS 38.473 §8.4.2 DL RRC Message Transfer (Successful operation)",
        "TS 38.473 §9.2.3.2 DL RRC MESSAGE TRANSFER",
        "TS 38.473 §9.2.2.10/§9.2.2.11 UE CONTEXT MODIFICATION",
        "TS 38.473 §9.3.1.23 Transaction ID"
      ],
      "code": [
        "F1AP/f1ap_du_interface_management.c:162-175",
        "F1AP/lib/f1ap_interface_management.c:1005-1009",
        "F1AP/f1ap_du_rrc_message_transfer.c:58-69",
        "F1AP/lib/f1ap_rrc_message_transfer.c:390-405",
        "F1AP/f1ap_du_ue_context_management.c:125-136"
      ]
    },
    "suggested_repro": [
      "Start CU/DU; complete F1 SETUP REQUEST→RESPONSE; attempt to inject/misroute an incoming F1 SETUP RESPONSE to CU and observe crash.",
      "Intercept a valid UE CONTEXT MODIFICATION REQUEST, alter RNTI in RRC payload, ensure CU does not crash and DU handles per spec.",
      "After UE release, send DL RRC MESSAGE TRANSFER to DU; verify context resolution and CU stability."
    ],
    "risks": [
      "Null dereference on missing UE context at CU",
      "Incorrect message direction handling at CU",
      "Insufficient validation of identifiers prior to dereference"
    ],
    "next_steps": [
      "Add explicit direction checks for inbound messages on CU task handlers",
      "Validate UE F1AP IDs and pdu->present/type before accessing payload",
      "Add unit/integration tests for misrouted F1 SETUP RESPONSE and malformed DL RRC",
      "Collect CU crash backtrace to pinpoint failing code path"
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.